In [1]:
import os
import pandas as pd
import librosa
import numpy as np

In [2]:
# Definir rutas de los archivos originales
audio_dir = 'audio_sample'
csv_path = 'all_data_updated.csv'

# Salida del csv procesado, para test se agrega "test" antes del nombre
output_path = 'output_file_50mfcc_4000Hz.csv'

# Nueva frecuencia de muestreo en Hz, librosa usa un filtro
new_sr = 4000

In [3]:
# Definición de archivos principales para cada set. 70 % para train por defecto.
df = pd.read_csv(csv_path)
audios_originales = df['file name']
test_audios_originales = audios_originales.sample(frac=0.7, random_state=42)

In [4]:
def extract_audio_embeddings(audio_path):
    """
    Función para extraer embeddings de un archivo de audio.
    Finalmente se trabaja con los MFCC, YAMNet no presentó buenos resultados.
    """
    y, sr = librosa.load(audio_path, sr=None)

    # Re muestreo
    if sr != new_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=new_sr)
        sr = new_sr

    # Usamos librosa para extraer embeddings. Por ejemplo, utilizando MFCC.
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)
    # Tomamos la media a lo largo del tiempo para obtener un vector de tamaño fijo.
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean

In [5]:
# Con segmentacion de archivos previo a la división
def process_audios(audio_dir, csv_path, output_path):
    # Leer el archivo CSV
    df = pd.read_csv(csv_path)
    # Crear una lista para almacenar los embeddings y las etiquetas
    embeddings = []
    labels = []
    files_name = []

    # Lo mismo, para separar set de test
    embeddings_t = []
    labels_t = []
    files_name_t = []
    
    # Recorrer el directorio de audios
    for root, dirs, files in os.walk(audio_dir):
        for file in files:
            if file.endswith(".wav"):  # Asumimos que los archivos de audio son .wav
                file_base_name = os.path.splitext(file)[0]

                if file_base_name[-1] == '5':
                    # Omitir los segment5, tienen aliasing
                    continue

                file_base_name = file_base_name[:-10]
                # Verificar si el archivo está en el CSV
                matching_rows = df[df['file name'].str.contains(file_base_name)]
                
                if not matching_rows.empty:
                    """
                    Se procesa el audio existente
                    """
                    audio_path = os.path.join(root, file)
                    embedding = extract_audio_embeddings(audio_path)

                    # Se determina a qué set corresponde
                    train_set_rows = test_audios_originales[test_audios_originales.str.contains(file_base_name)]

                    if not train_set_rows.empty:
                        # Set de Train

                        # Añadir embedding y etiqueta a las listas
                        embeddings.append(embedding)
                        labels.append(matching_rows.iloc[0]['queen status'])  # Asumimos que hay una columna 'queen status'
                        files_name.append(file)
                    else:
                        # Set de Test
                        embeddings_t.append(embedding)
                        labels_t.append(matching_rows.iloc[0]['queen status'])  # Asumimos que hay una columna 'queen status'
                        files_name_t.append(file)

    # Convertir listas a DataFrame

    # # Set de Train
    embeddings_df = pd.DataFrame(embeddings)
    embeddings_df['file name'] = files_name
    embeddings_df['label'] = labels
    
    # Guardar el DataFrame a un archivo
    embeddings_df.to_csv(output_path, index=False)

    # # Set de Train
    embeddings_df_t = pd.DataFrame(embeddings_t)
    embeddings_df_t['file name'] = files_name_t
    embeddings_df_t['label'] = labels_t
    
    # Guardar el DataFrame a un archivo
    embeddings_df_t.to_csv('test'+output_path, index=False)

In [6]:
# Ejecutar el procesamiento
process_audios(audio_dir, csv_path, output_path)